In [19]:
#importing libraries 
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [2]:
batch_s = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dimension = 256  # Latent dimensionality of the encoding space.
number_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra.txt'

In [3]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding = 'utf-8') as f:
    lines = f.read().split('\n')
lines[1]

'Go.\tMarche.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)'

In [4]:
for line in lines[: min(number_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [5]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [6]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [7]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [9]:
#Encoder:

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dimension, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [10]:
#DECODER ::
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dimension, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_lstm = LSTM(latent_dimension, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
form

In [20]:
model = Model([encoder_inputs ,decoder_inputs  ] , decoder_outputs )



In [21]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_s,
          epochs=15,
          validation_split=0.2)

Epoch 1/15
125/125 [==============================] - 46s 328ms/step - loss: 1.1356 - accuracy: 0.7372 - val_loss: 1.0169 - val_accuracy: 0.7114
Epoch 2/15
125/125 [==============================] - 33s 262ms/step - loss: 0.8203 - accuracy: 0.7778 - val_loss: 0.8282 - val_accuracy: 0.7669
Epoch 3/15
125/125 [==============================] - 32s 254ms/step - loss: 0.6490 - accuracy: 0.8166 - val_loss: 0.7073 - val_accuracy: 0.7953
Epoch 4/15
125/125 [==============================] - 36s 290ms/step - loss: 0.5855 - accuracy: 0.8312 - val_loss: 0.6403 - val_accuracy: 0.8088
Epoch 5/15
125/125 [==============================] - 45s 360ms/step - loss: 0.5315 - accuracy: 0.8444 - val_loss: 0.5959 - val_accuracy: 0.8258
Epoch 6/15
125/125 [==============================] - 44s 351ms/step - loss: 0.4961 - accuracy: 0.8542 - val_loss: 0.5599 - val_accuracy: 0.8365
Epoch 7/15
125/125 [==============================] - 40s 323ms/step - loss: 0.4675 - accuracy: 0.8619 - val_loss: 0.5363 - val_ac

In [22]:
model.save("seq2seq.h5")

In [26]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dimension,))
decoder_state_input_c = Input(shape=(latent_dimension,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [28]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [29]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [30]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

1/1 [==============================] - 0s 33ms/step
-
Input sentence: Go.
Decoded sentence: Allez !

1/1 [==============================] - 0s 32ms/step
-
Input sentence: Go.
Decoded sentence: Allez !

1/1 [==============================] - 0s 53ms/step
-
Input sentence: Go.
Decoded sentence: Allez !

1/1 [==============================] - 0s 38ms/step
-
Input sentence: Go.
Decoded sentence: Allez !

1/1 [==============================] - 0s 35ms/step
-
Input sentence: Hi.
Decoded sentence: Attendez !

1/1 [==============================] - 0s 40ms/step
-
Input sentence: Hi.
Decoded sentence: Attendez !

1/1 [==============================] - 0s 54ms/step
-
Input sentence: Run!
Decoded sentence: Auten !

1/1 [==============================] - 0s 36ms/step
-
Input sentence: Run!
Decoded sentence: Auten !

1/1 [==============================] - 0s 38ms/step
-
Input sentence: Run!
Decoded sentence: Auten !

1/1 [==============================] - 0s 41ms/step
-
Input sentence: Run!
Decoded

1/1 [==============================] - 0s 60ms/step
-
Input sentence: Run.
Decoded sentence: Coussez-vous.

1/1 [==============================] - 0s 52ms/step
-
Input sentence: Run.
Decoded sentence: Coussez-vous.

1/1 [==============================] - 0s 51ms/step
-
Input sentence: Run.
Decoded sentence: Coussez-vous.

1/1 [==============================] - 0s 57ms/step
-
Input sentence: Run.
Decoded sentence: Coussez-vous.

1/1 [==============================] - 0s 42ms/step
-
Input sentence: Run.
Decoded sentence: Coussez-vous.

1/1 [==============================] - 0s 44ms/step
-
Input sentence: Run.
Decoded sentence: Coussez-vous.

1/1 [==============================] - 0s 33ms/step
-
Input sentence: Run.
Decoded sentence: Coussez-vous.

1/1 [==============================] - 0s 51ms/step
-
Input sentence: Run.
Decoded sentence: Coussez-vous.

1/1 [==============================] - 0s 41ms/step
-
Input sentence: Who?
Decoded sentence: Tom est-il ?

1/1 [========================

1/1 [==============================] - 0s 38ms/step
-
Input sentence: Wow!
Decoded sentence: Laissez-moi partir.

1/1 [==============================] - 0s 40ms/step
-
Input sentence: Wow!
Decoded sentence: Laissez-moi partir.

1/1 [==============================] - 0s 34ms/step
-
Input sentence: Duck!
Decoded sentence: Laissez Tom !

1/1 [==============================] - 0s 40ms/step
-
Input sentence: Duck!
Decoded sentence: Laissez Tom !

1/1 [==============================] - 0s 37ms/step
-
Input sentence: Duck!
Decoded sentence: Laissez Tom !

1/1 [==============================] - 0s 42ms/step
-
Input sentence: Fire!
Decoded sentence: Laissez Tom !

1/1 [==============================] - 0s 52ms/step
-
Input sentence: Help!
Decoded sentence: Allez !

1/1 [==============================] - 0s 37ms/step
-
Input sentence: Hide.
Decoded sentence: Attendez !

1/1 [==============================] - 0s 36ms/step
-
Input sentence: Hide.
Decoded sentence: Attendez !

1/1 [================

1/1 [==============================] - 0s 37ms/step
-
Input sentence: Jump.
Decoded sentence: Allez-vous !

1/1 [==============================] - 0s 42ms/step
-
Input sentence: Stop!
Decoded sentence: Atrendez !

1/1 [==============================] - 0s 34ms/step
-
Input sentence: Stop!
Decoded sentence: Atrendez !

1/1 [==============================] - 0s 49ms/step
-
Input sentence: Stop!
Decoded sentence: Atrendez !

1/1 [==============================] - 0s 33ms/step
-
Input sentence: Wait!
Decoded sentence: Attendez !

1/1 [==============================] - 0s 38ms/step
-
Input sentence: Wait!
Decoded sentence: Attendez !

1/1 [==============================] - 0s 34ms/step
-
Input sentence: Wait!
Decoded sentence: Attendez !

1/1 [==============================] - 0s 43ms/step
-
Input sentence: Wait.
Decoded sentence: Attendez !

1/1 [==============================] - 0s 34ms/step
-
Input sentence: Wait.
Decoded sentence: Attendez !

1/1 [==============================] - 0s 35

1/1 [==============================] - 0s 30ms/step
-
Input sentence: Begin.
Decoded sentence: Allez !

1/1 [==============================] - 0s 32ms/step
-
Input sentence: Go on.
Decoded sentence: Va !

1/1 [==============================] - 0s 37ms/step
-
Input sentence: Go on.
Decoded sentence: Va !

1/1 [==============================] - 0s 37ms/step
-
Input sentence: Go on.
Decoded sentence: Va !

1/1 [==============================] - 0s 44ms/step
-
Input sentence: Hello!
Decoded sentence: Allez !

1/1 [==============================] - 0s 46ms/step
-
Input sentence: Hello!
Decoded sentence: Allez !

1/1 [==============================] - 0s 39ms/step
-
Input sentence: I see.
Decoded sentence: Je me suis senti bonte.

1/1 [==============================] - 0s 33ms/step
-
Input sentence: I see.
Decoded sentence: Je me suis senti bonte.

1/1 [==============================] - 0s 36ms/step
-
Input sentence: I try.
Decoded sentence: J'ai été présée.

1/1 [===========================

1/1 [==============================] - 0s 35ms/step
-
Input sentence: I won.
Decoded sentence: Je l'ai saus.

1/1 [==============================] - 0s 50ms/step
-
Input sentence: Oh no!
Decoded sentence: On vous !

1/1 [==============================] - 0s 47ms/step
-
Input sentence: Relax.
Decoded sentence: Allez-vous !

1/1 [==============================] - 0s 35ms/step
-
Input sentence: Relax.
Decoded sentence: Allez-vous !

1/1 [==============================] - 0s 34ms/step
-
Input sentence: Relax.
Decoded sentence: Allez-vous !

1/1 [==============================] - 0s 55ms/step
-
Input sentence: Relax.
Decoded sentence: Allez-vous !

1/1 [==============================] - 0s 37ms/step
-
Input sentence: Relax.
Decoded sentence: Allez-vous !

1/1 [==============================] - 0s 44ms/step
-
Input sentence: Relax.
Decoded sentence: Allez-vous !

1/1 [==============================] - 0s 101ms/step
-
Input sentence: Relax.
Decoded sentence: Allez-vous !

1/1 [===============

1/1 [==============================] - 0s 40ms/step
-
Input sentence: Relax.
Decoded sentence: Allez-vous !

1/1 [==============================] - 0s 49ms/step
-
Input sentence: Relax.
Decoded sentence: Allez-vous !

1/1 [==============================] - 0s 25ms/step
-
Input sentence: Relax.
Decoded sentence: Allez-vous !

1/1 [==============================] - 0s 42ms/step
-
Input sentence: Smile.
Decoded sentence: Coussiz-vous.

1/1 [==============================] - 0s 31ms/step
-
Input sentence: Smile.
Decoded sentence: Coussiz-vous.

1/1 [==============================] - 0s 31ms/step
-
Input sentence: Smile.
Decoded sentence: Coussiz-vous.

1/1 [==============================] - 0s 57ms/step
-
Input sentence: Sorry?
Decoded sentence: Es-tu en ure ?

1/1 [==============================] - 0s 44ms/step
-
Input sentence: Attack!
Decoded sentence: Attends !

1/1 [==============================] - 0s 30ms/step
-
Input sentence: Attack!
Decoded sentence: Attends !

1/1 [=============

1/1 [==============================] - 0s 28ms/step
-
Input sentence: Buy it.
Decoded sentence: Attendez !

1/1 [==============================] - 0s 37ms/step
-
Input sentence: Buy it.
Decoded sentence: Attendez !

1/1 [==============================] - 0s 31ms/step
-
Input sentence: Buy it.
Decoded sentence: Attendez !

1/1 [==============================] - 0s 34ms/step
-
Input sentence: Buy it.
Decoded sentence: Attendez !

1/1 [==============================] - 0s 33ms/step
-
Input sentence: Cheers!
Decoded sentence: Pous le poute !

1/1 [==============================] - 0s 29ms/step
-
Input sentence: Cheers!
Decoded sentence: Pous le poute !

1/1 [==============================] - 0s 32ms/step
-
Input sentence: Cheers!
Decoded sentence: Pous le poute !

1/1 [==============================] - 0s 28ms/step
-
Input sentence: Cheers!
Decoded sentence: Pous le poute !

1/1 [==============================] - 0s 36ms/step
-
Input sentence: Eat it.
Decoded sentence: Attendez !

1/1 [===

1/1 [==============================] - 0s 25ms/step
-
Input sentence: Get up.
Decoded sentence: Dégage !

1/1 [==============================] - 0s 31ms/step
-
Input sentence: Get up.
Decoded sentence: Dégage !

1/1 [==============================] - 0s 41ms/step
-
Input sentence: Get up.
Decoded sentence: Dégage !

1/1 [==============================] - 0s 33ms/step
-
Input sentence: Go now.
Decoded sentence: Va !

1/1 [==============================] - 0s 40ms/step
-
Input sentence: Go now.
Decoded sentence: Va !

1/1 [==============================] - 0s 35ms/step
-
Input sentence: Go now.
Decoded sentence: Va !

1/1 [==============================] - 0s 44ms/step
-
Input sentence: Got it!
Decoded sentence: Allez !

1/1 [==============================] - 0s 33ms/step
-
Input sentence: Got it!
Decoded sentence: Allez !

1/1 [==============================] - 0s 40ms/step
-
Input sentence: Got it!
Decoded sentence: Allez !

1/1 [==============================] - 0s 26ms/step
-
Input s